# Qassida : un générateur de poésie arabe avec LSTM sous Pytorch


*   A rédiger : Membres de l'équipe
*   Introduction + problématique 
*   Etapes de la réalistion



### Import des librairies :     
Nous commençons par l'import des librairies dont nous aurons besoin pour pouvoir implémenter notre solution
> * NumPy pour la gestion des calculs matriciels et vectoriels
> * PyTorch permet d'effectuer les calculs tensoriels nécessaires notamment pour l'apprentissage profond
> * Beautiful Soup pour le web scraping et la génération du dataset  

In [ ]:
import torch
from torch import nn
import torch.nn.functional as func
import numpy as np
import torch.nn.functional as F

### Chargement des données
Initialement nous exécutons un programme de web scraping qui fournit les données sous forme d'un fichier texte, pour effectuer l'entrainement après.

> Nous chargeons d'abord, le résultat du scraping à partir du fichier texte pour le pré-traiter 

In [ ]:
# Ouvrir le fichier texte lire les données sous forme de 'texte'

with open('poems_of_Darwish.txt', 'r', encoding="utf-8") as f:
  poem = f.read()

> Visualisons les 50 premiers caractères du texte :

In [ ]:
poem[:100]

'علي شاطء البحر بنت  وللبنت اهل\nوللاهل بيت  وللبيت نافذتان وباب\nوفي البحر بارجه تتسلي\nبصيد المشاه علي'

### Vectorisation
Dans ce qui suit, nous utilisons les structures 'dictionaries' de python pour convertir les caractères en un entier unique

In [ ]:
# Fonction 1 : char_to_int 
# Fonction 2 : int_to_char 

chars = tuple(set(poem))
int_to_char = dict(enumerate(chars))
char_to_int = {ch: ii for ii, ch in int_to_char.items()}

encoded = np.array([char_to_int[ch] for ch in poem])


> Visualisons l'encodage des 50 premiers caractères

In [ ]:
encoded[:50]

array([48, 33, 44, 55, 60, 53, 64, 39, 55, 53, 33, 41,  2, 15, 55, 41, 25,
       14, 55, 55, 16, 33, 33, 41, 25, 14, 55, 53, 26, 33, 11, 16, 33, 33,
       53, 26, 33, 55, 41, 44, 14, 55, 55, 16, 33, 33, 41, 44, 14, 55])

### Partie pré-traitement des données

Dans le modèle charRNN que nous voulant utiliser, le LSTM prend en entrée des données one-hot-encoded 
i.e un vecteur dont la taille est celle du vocabulaire et pour chaque caractère un '1' est placé à
la position du caractère dans ce vecteur, sinon '0'

In [ ]:
def one_hot(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

### Division en mini-batches d'entrainement

> On divise le vecteur des caractères encodés en séquence de taile seq_length selon le batch_size.


### Création des batches
> 1 - D'abord, il faut retrancher certain caractères pour avoir des batches de taille complète uniquement.
> 2 - Diviser le vecteur de caractères en N batches.
> 3 - Parcourir le vecteur qui est maintenant divisé en N sous-vecteurs pour avoir les mini-batches. 

### Entrainer et tester les Batched
> 1 - Créer 2 batches x,y où : x est le input batch et y est le training batch qui est exactement x mais décalé d'un seul caractère.

In [ ]:
def get_batches(arr, batch_size, seq_length):

    batch_size_total = batch_size * seq_length

    # Nombre total de batches que nous pouvons avoir
    n_batches = len(arr)//batch_size_total
    
    # Garder que les batches complets 
    arr = arr[:n_batches * batch_size_total]

    # Reshape en lignes de batch_size 
    arr = arr.reshape((batch_size, -1))
    
    # Itérer sur chaque séquence de caractères
    for n in range(0, arr.shape[1], seq_length):

        x = arr[:, n:n+seq_length]
        
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

### Visualiser la sortie 
> Visualisons ce que les batches sur 100 caractères de données encodées donne :

In [ ]:
batches = get_batches(encoded, 8, 50)
x, y = next(batches)

# Les 10 premiers éléments d'une séquence
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[48 33 44 55 60 53 64 39 55 53]
 [53 55 53 33  4 53 26 15 44 25]
 [12 55 53 33 16 24 14 55  3 44]
 [11 16  4 15 25 53 55 53 33 44]
 [55 40 25 55 53 33 15 16  2 55]
 [16 55 33 61 26 11 53 33 40 55]
 [53 55 40 53 39 26 53 55 44 15]
 [25 16 53 48 55 53 33  2 57 53]]

y
 [[33 44 55 60 53 64 39 55 53 33]
 [55 53 33  4 53 26 15 44 25 55]
 [55 53 33 16 24 14 55  3 44 55]
 [16  4 15 25 53 55 53 33 44 55]
 [40 25 55 53 33 15 16  2 55 16]
 [55 33 61 26 11 53 33 40 55 14]
 [55 40 53 39 26 53 55 44 15 16]
 [16 53 48 55 53 33  2 57 53 15]]


### Définir le modèle avec PyTorch
> On commence d'abord par définir les couches (layers) du modèle ensuite les fonctions (opérations), après on doit aussi écrire le code pour les opérations : forward (passer un caractère)


> __Remarque__ : on essaye d'utiliser les GPUs disponibles sinon, sélectionner le CPU

In [ ]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('L\'entrainement se fera sur GPU')
else: 
    print('Aucun GPU trouvé ')

L'entrainement se fera sur GPU


In [ ]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # Dictionnaires des caractères 
        self.chars = tokens
        self.int_to_char = dict(enumerate(self.chars))
        self.char_to_int = {ch: ii for ii, ch in self.int_to_char.items()}
        
        ## Définir le LSTM 
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        ## Définir la couche de dropout
        self.dropout = nn.Dropout(drop_prob)
        
        ## Définir la couche de sortie
        self.fc = nn.Linear(n_hidden, len(self.chars))
      
    
    def forward(self, x, hidden):
                
        ## Recevoir la sortie et le nouvel état 'hidden' depuis le LSTM
        r_output, hidden = self.lstm(x, hidden)
        
        ## Passer par une couche de dropout
        out = self.dropout(r_output)
        
        # Empiler (stacking) les sorties des LSTM 
        # Ici contiguous() c'est pour faire le reshape du vecteur 
        out = out.contiguous().view(-1, self.n_hidden)
        
        ## Faire passer x dans la couche entièrement connecté
        out = self.fc(out)
        
        # Retourner la sortie finale et le hidden state final
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initialiser le hidden state '''
        # Créations de 2 tensors de dimensions n_layers x batch_size x n_hidden,
        # initialiser à 0 pour le hidden state et cell state (cellule état) du LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden

In [ ]:
def train(net, data, epochs=5, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Entrainer le modèle
    
        Arguments
        ---------
        
        net: Le réseau charRNN
        data: Texte du dataset 
        epochs: Nombre d'epochs pour entrainer 
        batch_size: Nombre de mini-séquences par mini-batch (taille du batch)

        seq_length: Nombre de caractères par mini-batch
        lr: learning rate (taux d'apprentissage)
        clip: gradient clipping
        val_frac: Fraction de données à garder pour l'étape de validation
        print_every: Nombre d'étapes à compter jusqu'au prochain affichage du loss
    
    '''
    train_loss = 0.0
    valid_loss = 0.0
    valid_loss_min = np.Inf 


    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # Création des données de less et de validation
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialiser l'état hidden 
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            x = one_hot(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            h = tuple([each.data for each in h])

            # initialiser les gradients cumulés 
            net.zero_grad()
            
            # Recevoir la sortie 
            output, h = net(inputs, h)
            
            # Calculer la perte (loss) et effectuer une backpropagation
            loss = train_loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            # `clip_grad_norm` permet de controler l'explosion du gradient dans RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # statistiques du loss
            if counter % print_every == 0:
                
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    
                    x = one_hot(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = valid_loss =  criterion(output, targets.view(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                
                net.train() # Remettre en état d'entrainement après la phase de validation
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Etape: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))
                

### Entrainement
> __Remarque :__ L'entrainement sur Google Colab avec 50 epoch prend 4 heurs.

In [ ]:
# Définir le network (réseau)
n_hidden=128
n_layers=3

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(67, 128, num_layers=3, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=128, out_features=67, bias=True)
)


In [ ]:
batch_size = 32
seq_length = 100
n_epochs = 100

# Entrainer le modèle 
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

Epoch: 1/100... Etape: 10... Loss: 3.7269... Val Loss: 3.4927
Epoch: 1/100... Etape: 20... Loss: 3.1132... Val Loss: 3.0338
Epoch: 1/100... Etape: 30... Loss: 3.0434... Val Loss: 2.9763
Epoch: 1/100... Etape: 40... Loss: 3.0005... Val Loss: 2.9696
Epoch: 1/100... Etape: 50... Loss: 3.0019... Val Loss: 2.9648
Epoch: 1/100... Etape: 60... Loss: 2.9905... Val Loss: 2.9633
Epoch: 1/100... Etape: 70... Loss: 2.9908... Val Loss: 2.9623
Epoch: 1/100... Etape: 80... Loss: 2.9636... Val Loss: 2.9612
Epoch: 1/100... Etape: 90... Loss: 2.9928... Val Loss: 2.9621
Epoch: 1/100... Etape: 100... Loss: 3.0104... Val Loss: 2.9595
Epoch: 1/100... Etape: 110... Loss: 2.9801... Val Loss: 2.9594
Epoch: 1/100... Etape: 120... Loss: 2.9414... Val Loss: 2.9607
Epoch: 1/100... Etape: 130... Loss: 2.9675... Val Loss: 2.9582
Epoch: 1/100... Etape: 140... Loss: 2.9593... Val Loss: 2.9587
Epoch: 1/100... Etape: 150... Loss: 2.9761... Val Loss: 2.9566
Epoch: 1/100... Etape: 160... Loss: 2.9715... Val Loss: 2.9569
E

In [ ]:
model_name = 'Darwish_model.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(model_name, f)

In [ ]:
'''
with open('Darwish_model.net', 'rb') as f:
    if train_on_gpu:
            checkpoint = torch.load(f)
    else:        
        checkpoint = torch.load(f,map_location=torch.device('cpu'))

    
loaded = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])
'''

In [ ]:
def predict(net, char, h=None, top_k=None):
        
        x = np.array([[net.char_to_int[char]]])
        x = one_hot(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        h = tuple([each.data for each in h])
        
        out, h = net(inputs, h)

        # Calculer les probabilités du prochain caractère
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # passer au cpu
        
        # Garder les caractères avec les plus grandes probabiités
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # Choisir le prochain caractère à l'aide d'une fonction aléatoire
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # Retourner le code du caractère et le convertir en caractère  
        return net.int_to_char[char], h

In [ ]:
def sample(net, size, prime='The', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # mode d'évaluation
    
    # D'abord, traiter le premier 
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Passer le caractère précédent pour avoir le prochaine
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [ ]:
print(sample(net, 300, prime='السلام', top_k=5))